In [1]:
import random
import cadquery as cq
import numpy as np
import pyvista as pv
import numpy as np
from math import pi, sqrt, pow, acos, degrees, radians, sin, cos
from data import helper_temp as helper
from copy import deepcopy
import miniball
import pymeshfix as mf
from decimal import *
from scipy.stats import qmc
import math
import matplotlib.pyplot as plt
import itertools
import pandas as pd
import os

# Create parameter list

In [2]:
# define parameter ranges 
c_min, c_max = 10, 100
a_c_ratio_min, a_c_ratio_max = 0.1, 1
f_r0_min, f_r0_max = 0.75, 1
f_hp_min, f_hp_max = 0.75, 1.25
f_h0_min, f_h0_max = 0.75, 1.25

In [ ]:
# generate LHS
sampler = qmc.LatinHypercube(d = 5, optimization='random-cd')
samples = sampler.random(n=20)
samples[:5]

In [ ]:
# metric for quality of the samples
qmc.discrepancy(samples) 

In [ ]:
# scale samples to bounds
l_bounds = [c_min, a_c_ratio_min, f_r0_min, f_hp_min, f_h0_min]
u_bounds = [c_max, a_c_ratio_max, f_r0_max, f_hp_max, f_h0_max]
param_list = qmc.scale(samples, l_bounds, u_bounds)
param_list[:5]

In [ ]:
# plot 2-d cross sections of parameter space to visually inspect

# Calculate the ideal number of rows and columns
total_subplots = len(list(itertools.combinations(range(param_list.shape[1]), 2)))
ncols = math.ceil(math.sqrt(total_subplots))
nrows = math.ceil(total_subplots / ncols)

# Create subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols)
axes = axes.flatten()

# Plot each unique combination of columns
for i, (x_idx, y_idx) in enumerate(itertools.combinations(range(param_list.shape[1]), 2)):
    ax = axes[i]
    ax.scatter(param_list[:, x_idx], param_list[:, y_idx], s=10)
    ax.set_xlabel(f'Param {x_idx}')
    ax.set_ylabel(f'Param {y_idx}')
    ax.set_title(f'Param {x_idx} vs Param {y_idx}', fontsize=8)
    ax.set_box_aspect(1)
    # ax.set_aspect('equal', adjustable='datalim')  # Set aspect ratio to be equal

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
# plt.subplots_adjust(wspace=0.5, hspace=0.0)
plt.show()

param_list columns:
- col1 = c   
- col2 = a_c_ratio  
- col3 = f_r0   
- col4 = f_hp   
- col5 = f_h0  

In [7]:
# generate final parameter list, including diagnosed parameters
bullet_params = []
for i in range(len(param_list)):
    param = param_list[i]
    c = param[0]
    a = c*param[1]
    r0 = param[2]*a
    hp = r0*param[3]
    h0 = (hp/4)*param[4]
    record = [a, c, r0, hp, h0]
    bullet_params.append(record)

In [8]:
# convert to dataframe for easy reading and save as csv
colnames = ['a', 'c', 'r0', 'hp', 'h0']
df_bullet_params = pd.DataFrame(bullet_params, columns=colnames)

# save dataframe to csv
savepath = '/Users/josephko/research/ice3d/output/bullet_params.csv'
df_bullet_params.to_csv(savepath, index=False)

# Create bullets and save as STL

In [ ]:
# load bullet params and create bullets using cadquery
filepath = '/Users/josephko/research/ice3d/output/bullet_params.csv'
bullet_params = pd.read_csv(filepath)
bullet_params

In [10]:
# define function to create bullet using cadquery
def create_bullet(a, c, hp):
    # create pyramid
    n_pyr = 6
    theta = 90 - np.degrees(np.arctan(hp/a))
    pyramid = cq.Workplane().polygon(n_pyr, 2*a).extrude(hp, taper=theta)

    # create cylinder 
    n_cyl = 6
    cylinder = cq.Workplane().polygon(n_cyl, 2*a).extrude(2*c)

    # move pyramid up on top of cylinder
    pyramid = pyramid.translate((0,0,2*c))

    # create bullet (union)
    bullet = cylinder.union(pyramid)
    return bullet

# def create_bullet(a, c, hp):
#     # create pyramid
#     n_pyr = 6
#     theta = 90 - np.degrees(np.arctan(hp/a))
#     pyramid = cq.Workplane().polygon(n_pyr, 2*a).extrude(-hp, taper=theta)

#     # create cylinder 
#     n_cyl = 6
#     cylinder = cq.Workplane().polygon(n_cyl, 2*a).extrude(c)

#     # create bullet (union)
#     bullet = cylinder.union(pyramid)

#     # shift bullet up so tip is at z=0
#     bullet = bullet.translate((0,0,hp))

#     return bullet


# create bullet for each parameter set
bullets = []
output_dir = '/Users/josephko/research/ice3d/output/bullet_stl'
for index, row in bullet_params.iterrows():
    a, c, hp = row['a'], row['c'], row['hp']
    bullet = create_bullet(a, c, hp)
    # save bullet
    filename = f'bullet_{index:04d}.stl'
    filepath = os.path.join(output_dir, filename)
    # print(filepath)
    cq.exporters.export(bullet, filepath)
    # print(index)
    # print(type(index))



# Create full rosettes

In [ ]:
bullet_params.head()

## Function to create bullet rosette

In [6]:
# define function to create bullet rosette
def create_rosette(a, c, r0, hp, h0, index, bullet_dir):
    n_arms = random.randint(4, 12)
    # create sphere
    sphere = pv.Sphere(radius=r0, center=(0, 0, 0), direction=(0, 0, 1), 
                        theta_resolution=30, phi_resolution=20, start_theta=0, 
                        end_theta=360, start_phi=0, end_phi=180)
    sphere = sphere.triangulate()

    # create outer shell to "place" bullets on
    r_outer = hp/2 + c - h0 + r0
    print(f'r_outer: {r_outer}')
    if n_arms == 2: # line
        outer_shell = pv.Line(pointa=(-r_outer, 0.0, 0.0), 
                            pointb=(r_outer, 0.0, 0.0), resolution=1)
        outer_coords = outer_shell.points
    elif n_arms == 4: # tetrahedron
        outer_shell = pv.Tetrahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
        outer_coords = outer_shell.points
    elif n_arms == 6: # octahedron
        outer_shell = pv.Octahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
        outer_coords = outer_shell.points
    elif n_arms == 8: # cube
        # Note: this may not be the optimal solution for n=8, check later
        l  = (2*r_outer)/(3**(1/2))
        outer_shell = pv.Cube(center=(0.0, 0.0, 0.0), x_length=l, 
                            y_length=l, z_length=l)
        outer_coords = outer_shell.points
    else: 
        # Modified fibbonaci lattice 
        # Source: http://extremelearning.com.au/how-to-evenly-distribute-points-on-a-sphere-more-effectively-than-the-canonical-fibonacci-lattice/
        epsilon = 0.33
        goldenRatio = (1 + 5**0.5)/2
        i = np.arange(0, n_arms) 
        theta = 2 *pi * i / goldenRatio
        phi = np.arccos(1 - 2*(i+epsilon)/(n_arms-1+2*epsilon))
        x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
        outer_coords = r_outer*(np.column_stack((x, y, z)))
    # load bullet arm from STL that were created with cadquery
    bullet_filename = f'bullet_{index:04}.stl'
    bullet_path = os.path.join(bullet_dir, bullet_filename)
    print(bullet_path)
    bullet = pv.read(bullet_path)
    # rotate bullet in "correct" default orientation
    center_temp = bullet.center
    center_temp_negative = [-num for num in center_temp]
    bullet.translate(center_temp_negative, inplace=True)
    bullet.rotate_x(180, inplace=True)
    bullet.translate(center_temp, inplace=True)
    bullet = bullet.triangulate()
    # copy, translate, and rotate bullets
    bullets = {} # save bullets in nested dictionary
    bullet_center_default = bullet.center
    for i in range(len(outer_coords)):
        bullet_entry = {}
        pt = outer_coords[i]
        print(f'pt: {pt}')
        #rotate 
        bullet_center = bullet_center_default # center before any hollowing
        print(f'bullet center: {bullet_center}')
        cross_prod = np.cross(bullet_center, pt)
        theta = degrees( acos (np.dot(bullet_center, pt) / (np.linalg.norm(bullet_center)*np.linalg.norm(pt))) )
        if not np.any(np.array(cross_prod)): # if all zeros
            if theta == 180:
                bullet_rot = bullet.rotate_x(180)
            else:
                bullet_rot = bullet
            new_center = bullet_rot.center
        else:
            rads = radians(theta)
            transform_mat = helper.rotate_axis_angle(cross_prod, rads)
            rot_mat = transform_mat[:3, :3]
            new_center = rot_mat @ bullet_center # matrix multiply to rotate
            bullet_rot = bullet.transform(transform_mat, inplace=False)
        # translate
        translate_vector = pt - new_center
        bullet_final = bullet_rot.translate(translate_vector, inplace=False)
        print(f'new center = {new_center}')
        print(f'bullet_final center = {bullet_final.center}')
        # add bullet attributes and mesh to dictionary
        bullet_entry['bullet_rot_center'] = new_center
        bullet_entry['mesh'] = bullet_final.triangulate()
        bullet_entry['xy_scale_factor'] = 1.0
        bullet_entry['z_scale_factor'] = 1.0
        bullet_entry['anchor_point'] = pt
        bullets[i] = bullet_entry

    # # perform booolean union 
    # rosette = sphere
    # for i in range(n_arms):
    #     print(i)
    #     b = bullets[i]
    #     bullet_mesh = b['mesh'].triangulate()
    #     rosette = rosette.boolean_union(bullet_mesh).triangulate()
    # return rosette
    return sphere, bullets

## Create rosettes

In [2]:
output_dir = '/Users/josephko/research/ice3d/output/rosette_stl'
bullet_dir = '/Users/josephko/research/ice3d/output/bullet_stl'
bullet_params = pd.read_csv('/Users/josephko/research/ice3d/output/bullet_params.csv')
bullet_params.head()

,a,c,r0,hp,h0
0,22.175942,45.775049,22.080469,23.100185,6.884353
1,6.906007,11.129418,6.324192,6.960184,1.896451
2,4.774335,30.872733,4.067800,3.330811,0.916684
3,6.703981,19.432683,6.500955,5.898020,1.396309
4,31.650980,75.316580,27.588879,34.108545,9.788097


In [ ]:
for index, row in bullet_params.iterrows():
    a, c, r0, hp, h0 = row['a'], row['c'], row['r0'], row['hp'], row['h0']
    rosette = create_rosette(a, c, r0, hp, h0, index, bullet_dir)
    # save rosette as stl
    rosette_filename = f'rosette_{index:04d}.stl'
    rosette_filepath = os.path.join(output_dir, rosette_filename)
    rosette.save(rosette_filepath)

In [3]:
row = bullet_params.iloc[0]
a, c, r0, hp, h0 = row['a'], row['c'], row['r0'], row['hp'], row['h0']
print(a, c, r0, hp, h0)


22.175942110287863 45.77504933721399 22.080468623453847 23.100185493961774 6.884352668479022


In [4]:
print(hp/2, c, h0, r0)

11.550092746980887 45.77504933721399 6.884352668479022 22.080468623453847


In [8]:
sphere, bullets = create_rosette(a, c, r0, hp, h0, 0, bullet_dir)
pl = pv.Plotter()
pl.add_mesh(sphere)
for i in range(len(bullets)):
    pl.add_mesh(bullets[i]['mesh'])
pl.show()

r_outer: 72.52125803916971
/Users/josephko/research/ice3d/output/bullet_stl/bullet_0000.stl
pt: [25.12148169  0.         68.03119891]
bullet center: [0.0, 0.0, 55.77772521972656]
new center = [19.3214947   0.         52.32432009]
bullet_final center = [28.586425304412842, 0.0, 71.87209367752075]
pt: [-35.34187334 -32.37605103  54.42495913]
bullet center: [0.0, 0.0, 55.77772521972656]
new center = [-27.18222702 -24.9011466   41.85945607]
bullet_final center = [-40.21648848056793, -36.84159964323044, 61.41359519958496]
pt: [ 5.24056151 59.71349541 40.81871935]
bullet center: [0.0, 0.0, 55.77772521972656]
new center = [ 4.03063333 45.92698788 31.39459206]
bullet_final center = [5.96337890625, 65.36141204833984, 46.4487419128418]
pt: [ 40.90055196 -53.34752733  27.21247956]
bullet center: [0.0, 0.0, 55.77772521972656]
new center = [ 31.45753136 -41.03077913  20.92972804]
bullet_final center = [46.54186189174652, -60.70561742782593, 30.965829610824585]
pt: [-70.14452989  12.40757967  13.606

Widget(value='<iframe src="http://localhost:57875/index.html?ui=P_0x312d43fe0_1&reconnect=auto" class="pyvista…

In [6]:
n_arms = random.randint(4, 12)
sphere = pv.Sphere(radius=r0, center=(0, 0, 0), direction=(0, 0, 1), 
                    theta_resolution=30, phi_resolution=20, start_theta=0, 
                    end_theta=360, start_phi=0, end_phi=180)
sphere = sphere.triangulate()

# create outer shell to "place" bullets on
r_outer = hp/2 + c - h0 + r0
outer_sphere = pv.Sphere(radius=r_outer, center=(0, 0, 0), direction=(0, 0, 1), 
                theta_resolution=30, phi_resolution=20, start_theta=0, 
                end_theta=360, start_phi=0, end_phi=180)
if n_arms == 2: # line
    outer_shell = pv.Line(pointa=(-r_outer, 0.0, 0.0), 
                        pointb=(r_outer, 0.0, 0.0), resolution=1)
    outer_coords = outer_shell.points
elif n_arms == 4: # tetrahedron
    outer_shell = pv.Tetrahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
    outer_coords = outer_shell.points
elif n_arms == 6: # octahedron
    outer_shell = pv.Octahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
    outer_coords = outer_shell.points
elif n_arms == 8: # cube
    # Note: this may not be the optimal solution for n=8, check later
    l  = (2*r_outer)/(3**(1/2))
    outer_shell = pv.Cube(center=(0.0, 0.0, 0.0), x_length=l, 
                        y_length=l, z_length=l)
    outer_coords = outer_shell.points
else: 
    # Modified fibbonaci lattice 
    # Source: http://extremelearning.com.au/how-to-evenly-distribute-points-on-a-sphere-more-effectively-than-the-canonical-fibonacci-lattice/
    epsilon = 0.33
    goldenRatio = (1 + 5**0.5)/2
    i = np.arange(0, n_arms) 
    theta = 2 *pi * i / goldenRatio
    phi = np.arccos(1 - 2*(i+epsilon)/(n_arms-1+2*epsilon))
    x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
    outer_coords = r_outer*(np.column_stack((x, y, z)))

In [ ]:
# load bullet arm from STL that were created with cadquery
index = 0
bullet_filename = f'bullet_{index:04}.stl'
bullet_path = os.path.join(bullet_dir, bullet_filename)
print(bullet_path)
bullet = pv.read(bullet_path)
# rotate bullet in "correct" defautl orientation
center_temp = bullet.center
center_temp_negative = [-num for num in center_temp]
bullet.translate(center_temp_negative, inplace=True)
bullet.rotate_x(180, inplace=True)
bullet.translate(center_temp, inplace=True)
# copy, translate, and rotate bullets
bullets = {} # save bullets in nested dictionary
bullet_center_default = bullet.center

In [ ]:
bullet.plot()

In [9]:
for i in range(len(outer_coords)):
    bullet_entry = {}
    pt = outer_coords[i]
    #rotate 
    bullet_center = bullet_center_default # center before any hollowing
    cross_prod = np.cross(bullet_center, pt)
    theta = degrees( acos (np.dot(bullet_center, pt) / (np.linalg.norm(bullet_center)*np.linalg.norm(pt))) )
    if not np.any(np.array(cross_prod)): # if all zeros
        if theta == 180:
            bullet_rot = bullet.rotate_x(180)
        else:
            bullet_rot = bullet
        new_center = bullet_rot.center
    else:
        rads = radians(theta)
        transform_mat = helper.rotate_axis_angle(cross_prod, rads)
        rot_mat = transform_mat[:3, :3]
        new_center = rot_mat @ bullet_center # matrix multiply to rotate
        bullet_rot = bullet.transform(transform_mat, inplace=False)
    # translate
    translate_vector = pt - new_center
    bullet_final = bullet_rot.translate(translate_vector, inplace=False)
    # add bullet attributes and mesh to dictionary
    bullet_entry['bullet_rot_center'] = new_center
    bullet_entry['mesh'] = bullet_final.triangulate()
    bullet_entry['xy_scale_factor'] = 1.0
    bullet_entry['z_scale_factor'] = 1.0
    bullet_entry['anchor_point'] = pt
    bullets[i] = bullet_entry

        # # perform booolean union 
        # rosette = sphere
        # for i in range(n_arms):
        #     b = bullets[i]
        #     bullet_mesh = b['mesh'].triangulate()
        #     rosette = rosette.boolean_union(bullet_mesh).triangulate()

In [ ]:
pl = pv.Plotter()
pl.add_mesh(sphere)
for i in range(len(bullets)):
    pl.add_mesh(bullets[i]['mesh'])
pl.show()

In [12]:
rosette = sphere

In [ ]:
# i = 0
i=0
b = bullets[i]
bullet_mesh = b['mesh'].triangulate()
rosette = rosette.boolean_union(bullet_mesh).triangulate()

a, c, r0, hp, h0 = 52.707552592700985 60.79304822413943 43.9651533204692 50.02875758839475 15.533126444390852

In [ ]:
from data.rosette_temp import Rosette

ros = Rosette(52, 60, 43, 15, 50, 4)
pl = ros.plot()
pl.show()

In [ ]:
a, c, r0, h0, hp, n_arms = 1, 4, 1, 0.5, 1, 6
# create bullet arm
cyl = pv.Cylinder(center=(0.0, 0.0, c+hp), direction=(0.0, 0.0, -1.0), 
                radius=a, height=2*c, resolution=6, capping=True)
pyr = pv.Cone(center=(0.0, 0.0, hp/2), direction=(0.0, 0.0, -1.0), 
            height=hp, radius=a, capping=True, angle=None, resolution=6)
cyl = cyl.triangulate()
pyr = pyr.triangulate()
cyl_pts = cyl.points
cyl_pts[abs(cyl_pts)<1e-10]=0.0 # replace small values with zeros
cyl.points = cyl_pts
pyr_pts = pyr.points
pyr_pts[abs(pyr_pts)<1e-10]=0.0 # replace small values with zeros
pyr.points = pyr_pts
bullet = cyl.boolean_union(pyr).triangulate()
pt_dist = np.linalg.norm(bullet.points, axis=1)
bullet.plot()
# # print('this is a test')
# tip_pt_index = np.argmin(pt_dist) # index of the tip point in bullet.points
# self.bullet_center_default = bullet.center # for testing
# self.bullet_default = bullet

In [ ]:
bullet.center

In [ ]:
bullet_cq = pv.read('/Users/josephko/research/ice3d/output/bullet_stl/bullet_0001.stl')
bullet_cq.plot()

In [ ]:
# rotate bullet in "correct" defautl orientation
bullet_cq_rot = bullet_cq.copy()
center_temp = bullet_cq_rot.center
center_temp_negative = [-num for num in center_temp]
bullet_cq_rot.translate(center_temp_negative, inplace=True)
bullet_cq_rot.rotate_x(180, inplace=True)
bullet_cq_rot.translate(center_temp, inplace=True)
bullet_cq_rot.plot()

In [ ]:
bullet_cq_rot.center

# Cadquery to pyvista pipeline: single rosette

In [19]:
def create_bullet(a, c, hp):
    # create pyramid
    n_pyr = 6
    theta = 90 - np.degrees(np.arctan(hp/a))
    pyramid = cq.Workplane().polygon(n_pyr, 2*a).extrude(-hp, taper=theta)

    # create cylinder 
    n_cyl = 6
    cylinder = cq.Workplane().polygon(n_cyl, 2*a).extrude(2*c)

    # create bullet (union)
    bullet = cylinder.union(pyramid)

    # shift bullet up so tip is at z=0
    bullet = bullet.translate((0,0,hp))

    return bullet

In [20]:
a, c, hp = 1, 4, 1
bullet = create_bullet(a, c, hp)
cq.exporters.export(bullet, '/Users/josephko/research/ice3d/output/bullet.stl')

In [6]:
r0, h0, n_arms = 1, 0.5, 4
sphere = pv.Sphere(radius=r0, center=(0, 0, 0), direction=(0, 0, 1), 
                    theta_resolution=30, phi_resolution=20, start_theta=0, 
                    end_theta=360, start_phi=0, end_phi=180)
sphere = sphere.triangulate()

# create outer shell to "place" bullets on
r_outer = hp/2 + c - h0 + r0
outer_sphere = pv.Sphere(radius=r_outer, center=(0, 0, 0), direction=(0, 0, 1), 
                theta_resolution=30, phi_resolution=20, start_theta=0, 
                end_theta=360, start_phi=0, end_phi=180)
if n_arms == 2: # line
    outer_shell = pv.Line(pointa=(-r_outer, 0.0, 0.0), 
                        pointb=(r_outer, 0.0, 0.0), resolution=1)
    outer_coords = outer_shell.points
elif n_arms == 4: # tetrahedron
    outer_shell = pv.Tetrahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
    outer_coords = outer_shell.points
elif n_arms == 6: # octahedron
    outer_shell = pv.Octahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
    outer_coords = outer_shell.points
elif n_arms == 8: # cube
    # Note: this may not be the optimal solution for n=8, check later
    l  = (2*r_outer)/(3**(1/2))
    outer_shell = pv.Cube(center=(0.0, 0.0, 0.0), x_length=l, 
                        y_length=l, z_length=l)
    outer_coords = outer_shell.points
else: 
    # Modified fibbonaci lattice 
    # Source: http://extremelearning.com.au/how-to-evenly-distribute-points-on-a-sphere-more-effectively-than-the-canonical-fibonacci-lattice/
    epsilon = 0.33
    goldenRatio = (1 + 5**0.5)/2
    i = np.arange(0, n_arms) 
    theta = 2 *pi * i / goldenRatio
    phi = np.arccos(1 - 2*(i+epsilon)/(n_arms-1+2*epsilon))
    x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
    outer_coords = r_outer*(np.column_stack((x, y, z)))

In [ ]:
sphere.plot()

In [8]:
 # create outer shell to "place" bullets on
r_outer = hp/2 + c - h0 + r0
if n_arms == 2: # line
    outer_shell = pv.Line(pointa=(-r_outer, 0.0, 0.0), 
                        pointb=(r_outer, 0.0, 0.0), resolution=1)
    outer_coords = outer_shell.points
elif n_arms == 4: # tetrahedron
    outer_shell = pv.Tetrahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
    outer_coords = outer_shell.points
elif n_arms == 6: # octahedron
    outer_shell = pv.Octahedron(radius=r_outer, center=(0.0, 0.0, 0.0))
    outer_coords = outer_shell.points
elif n_arms == 8: # cube
    # Note: this may not be the optimal solution for n=8, check later
    l  = (2*r_outer)/(3**(1/2))
    outer_shell = pv.Cube(center=(0.0, 0.0, 0.0), x_length=l, 
                        y_length=l, z_length=l)
    outer_coords = outer_shell.points
else: 
    # Modified fibbonaci lattice 
    # Source: http://extremelearning.com.au/how-to-evenly-distribute-points-on-a-sphere-more-effectively-than-the-canonical-fibonacci-lattice/
    epsilon = 0.33
    goldenRatio = (1 + 5**0.5)/2
    i = np.arange(0, n_arms) 
    theta = 2 *pi * i / goldenRatio
    phi = np.arccos(1 - 2*(i+epsilon)/(n_arms-1+2*epsilon))
    x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
    outer_coords = r_outer*(np.column_stack((x, y, z)))

In [ ]:
outer_coords

In [ ]:
# import bullet
bullet = pv.read('/Users/josephko/research/ice3d/output/bullet.stl')
bullet.plot()

In [ ]:
cyl = pv.Cylinder(center=(0.0, 0.0, c+hp), direction=(0.0, 0.0, -1.0), 
                radius=a, height=2*c, resolution=6, capping=True)
pyr = pv.Cone(center=(0.0, 0.0, hp/2), direction=(0.0, 0.0, -1.0), 
            height=hp, radius=a, capping=True, angle=None, resolution=6)
cyl = cyl.triangulate()
pyr = pyr.triangulate()
bullet_pv = cyl.boolean_union(pyr).triangulate()
bullet_pv.plot()

In [ ]:
bullet_pv.center

In [ ]:
bullet_center_default = bullet.center

In [ ]:
bullet_center_default

In [ ]:
bullets = {}
for i in range(len(outer_coords)):
    bullet_entry = {}
    pt = outer_coords[i]
    bullet_center = bullet_center_default
    